In [1]:
pip install -q nbimporter nbformat python-dotenv

In [2]:
pip install langchain langgraph langchain-core langchain-google-genai requests pytesseract z3-solver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is t

In [28]:
import os
from typing import Optional, List
import time
from langchain_core.tools import tool
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import StateGraph, START, MessagesState
from pydantic import BaseModel, Field
from typing_extensions import TypedDict
from typing import Literal
from openai import OpenAI
from langchain_core.messages import SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from bs4 import BeautifulSoup
from langchain_core.tools import tool
import traceback
import sys
import io
import requests
from PIL import Image
import pytesseract
from z3 import *
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["LANGSMITH_API_KEY"] = "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"
os.environ["LANGSMITH_TRACING_V2"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "pr-authorized-cheese-72"


REACT_SYS_PROMPT = """You are an intelligent agent who can:
- Given a math problem and optional feedback, generate valid Python Z3 code that proves the theorem.
- You must use the prove method from Z3 so that you would get counterexamples.
- If given feedback about proof failure or counterexamples, adjust the axioms or theorem accordingly.
- Only output the final Z3 code snippet, no explanations.
- You must execute the code before outputting so that you'd be sure it is right. Don't output any code without first running it using Z3.
- If you are sure that what the user wants is not right state that clearly.
- Do not implement anything other than what the user asked for. For example if the user said less than don't put less than or equal.
- DO NOT SAY THAT SOMETHING CANNOT BE PROVED BY Z3. ANY TRUE STATEMENT CAN BE BROKEN DOWN TO SMALLER ELEMENTS AND BE PROVED BY Z3.
- Take your time and until you haven't proved the statement continue using the tools.
"""


@tool
def run_z3_proof(code: str) -> str:
    """
    Execute given Python Z3 code and detect if the theorem was proved or not.

    Returns a simple result string indicating success or failure with details.
    """
    old_stdout = sys.stdout
    sys.stdout = mystdout = io.StringIO()
    try:
        exec(code, {})
        output = mystdout.getvalue()
    except Exception:
        output = traceback.format_exc()
    finally:
        sys.stdout = old_stdout

    low = output.lower()
    if "proved" in low:
        return "PROVED"
    elif "counterexample" in low or "failed" in low or "assertion" in low or "error" in low:
        return f"FAILED: {output.strip()}"
    else:
        return f"UNKNOWN RESULT: {output.strip()}"

@tool
def select_axioms(problem_statement: str) -> str:
    """
    Ask the LLM to list relevant mathematical axioms for the given problem.

    Args:
        problem_statement: A math problem description in text.

    Returns:
        A text list of axioms or assumptions that help prove the problem.
    """
    prompt = (
        f"You are a mathematician. Given the following problem:\n\n"
        f"{problem_statement}\n\n"
        f"List the key mathematical axioms or assumptions relevant to solving this problem. "
        f"Return the axioms as a concise numbered list."
    )
    llm = ChatOpenAI(
        api_key=os.getenv("GEMINI_API_KEY"), temperature=0.1, base_url="https://api.metisai.ir/openai/v1", model=os.environ['MODEL_NAME'])

    response = llm.invoke([{"role": "user", "content": prompt}])

    return response.content

class Z3Solver:
    def __init__(self):
        self.api_token =  os.getenv("GEMINI_API_KEY")
        self.model_name = os.environ['MODEL_NAME']
        self.tools = [run_z3_proof, select_axioms]

        self.llm = ChatGoogleGenerativeAI(
            api_key=self.api_token, temperature=0.1, model=self.model_name)
        self.model_with_tools = self.llm.bind_tools(self.tools)
        self.react_graph = self.build_graph()

    def agent_node(self, state: MessagesState) -> MessagesState:
        sys_msg = SystemMessage(
            content=REACT_SYS_PROMPT)
        messages = [sys_msg] + state["messages"]
        response = self.model_with_tools.invoke(messages)
        return {"messages": state["messages"] + [response]}

    def build_graph(self):
        tools_node = ToolNode(tools=self.tools)
        builder = StateGraph(MessagesState)

        builder.add_node("agent_node", self.agent_node)
        builder.add_node("tools", tools_node)

        builder.add_edge(START, "agent_node")
        builder.add_conditional_edges("agent_node", tools_condition)
        builder.add_edge("tools", "agent_node")

        return builder.compile()

    def capture_the_flag(self, question):
        inputs = {
            "messages": [
                {"role": "user", "content": question}
            ]
        }

        response = self.react_graph.invoke(inputs)
        last_msg = response["messages"][-1].content
        if isinstance(last_msg, list):
            last_msg = last_msg[-1]

        return response["messages"]

In [ ]:
os.environ['MODEL_NAME'] = "gemini-2.5-flash"
x = Z3Solver()
res = x.capture_the_flag(
    "Make sure you prove this problem: Prove Collatz conjecture. Do your best to prove it and don't say that you can not prove it using Z3.")
# res = x.capture_the_flag(
#     "Make sure you prove this problem: Prove given two numbers greater than 1, the sum of their inverse is less than 2.")
for msg in res:
    msg.pretty_print()

In [30]:
from z3 import *

x, y = Reals('x y')

# Axioms
s = Solver()
s.add(x > 1)
s.add(y > 1)

# Theorem to prove: 1/x + 1/y < 2
# In Z3, to prove A, we assert not A and check for unsatisfiability.
# So, we assert not (1/x + 1/y < 2), which is 1/x + 1/y >= 2
s.add(Not(1/x + 1/y < 2))

# Check if the assertions are unsatisfiable
if s.check() == unsat:
    print("Proved")
else:
    print("Failed to prove")
    print(s.model())

Proved
